# Azure OpenAI 画像生成デモ (DALL-E 3 & GPT-image シリーズ)

このノートブックでは Azure OpenAI の **DALL-E 3** および **GPT-image シリーズ (例: gpt-image-1)** を用いて以下を実行します：

1. テキストプロンプトから画像を生成
2. スタイル・品質・サイズパラメータの実験 (DALL-E 3)
3. 画像の保存とバッチ生成
4. プロンプトエンジニアリングのベストプラクティス

---
### 🖼 Azure OpenAI 画像生成 API 概要
Azure OpenAI の画像生成は **DALL-E 3** と **GPT-image** 系モデルが利用可能です。DALL-E 3 は強力なビジュアル表現とスタイル/品質パラメータを提供し、GPT-image シリーズは軽量で高速・コスト効率と自然なテキスト整合性を重視した汎用画像生成/編集基盤です。

#### 主なモデル差分（2025 年時点）
| 項目 | DALL-E 3 | GPT-image-1 |
|------|----------|-------------|
| 目的 | 高精細クリエイティブ生成 | 汎用/高速生成・省コスト |
| プロンプト理解 | ★★★ (高度な詳細化) | ★★☆ (効率重視) |
| revised_prompt | あり (自動詳細化) | なし (入力忠実) |
| スタイル指定 | `style=vivid/natural` | なし (自然傾向) |
| 品質指定 | `quality=standard/hd` | なし (単一品質) |
| サイズオプション | 1024x1024 / 1024x1792 / 1792x1024 | 主に 1024x1024 (※プレビューで拡張予定) |
| 生成枚数 (`n`) | 1 固定 | 複数可 (デプロイ設定依存) |
| コスト | 高 | 低～中 |
| 推奨用途 | マーケ/広告/高付加価値素材 | プロトタイプ/大量生成/自動化パイプライン |

#### 代表パラメータ (API 呼び出し)
| パラメータ | 意味 | 備考 |
|------------|------|------|
| `model` | デプロイメント名 | 例: `dall-e-3`, `gpt-image-1` |
| `size` | 出力画像サイズ | DALL-E 3 のみ複数サイズ。GPT-image は標準サイズ中心 |
| `quality` | 画質 (standard/hd) | DALL-E 3 のみ |
| `style` | vivid / natural | DALL-E 3 のみ |
| `revised_prompt` | 生成時補正後プロンプト | DALL-E 3 のみ返却 |

#### 使い分けガイド
- 高品質な広告/ブランド素材 → **DALL-E 3 + hd (必要時)**
- 大量生成 / コスト最適化 → **GPT-image-1 (standard品質相当)**
- 自然 / 写真調 → DALL-E 3 の `style=natural` または GPT-image-1
- 鮮やか / アート調 → DALL-E 3 の `style=vivid`

#### ベストプラクティス（概要）
- 主要要素 (被写体 / 背景 / スタイル / 光源 / 色 / 雰囲気) を構造化列挙
- 長すぎる記述は重要語に整理（DALL-E 3 は詳細化機構あり）
- PoC は GPT-image → 本番候補のみ DALL-E 3 へ昇格
- 生成結果とプロンプト(revised含む)をメタデータ保存し再現性担保

> 以下の各セクション見出しに続くコードの目的概要を記載しています。

## 1. 環境セットアップ

**概要**: 画像生成に必要なライブラリ (openai / Pillow / requests) と Azure OpenAI のエンドポイント・APIキー・DALL-E デプロイ名を `.env.dev` から読み込みクライアント初期化を行います。秘密情報は環境変数経由で扱い、後続の画像生成関数で再利用できる状態を作ります。

### 環境変数の設定

`.env.dev` に以下を設定します。DALL-E 3 と GPT-image を併用するため、それぞれのデプロイ名を分離し用途に応じ切替できる構成にします。

例:
```
AZURE_OPENAI_ENDPOINT=https://your-resource.openai.azure.com/
AZURE_OPENAI_API_KEY=your-api-key-here
AZURE_OPENAI_API_VERSION=2024-02-01
DALLE3_DEPLOYMENT_NAME=dall-e-3
GPT_IMAGE_DEPLOYMENT_NAME=gpt-image-1
```

- 片方のみ運用開始する場合は不要な方を後で追加可能。
- GPT-image 系は style/quality/revised_prompt を返さない軽量モデルのためパラメータ分岐を後続関数で行います。

In [7]:
# 必要なライブラリのインストール確認
import sys
print(f"Python バージョン: {sys.version}")

try:
    import openai
    print(f"✅ openai バージョン: {openai.__version__}")
except ImportError:
    print("❌ openai ライブラリがインストールされていません")
    print("インストール: pip install openai")

try:
    from PIL import Image
    print(f"✅ Pillow (PIL) インストール済み")
except ImportError:
    print("❌ Pillow ライブラリがインストールされていません")
    print("インストール: pip install Pillow")

try:
    import requests
    print(f"✅ requests インストール済み")
except ImportError:
    print("❌ requests ライブラリがインストールされていません")
    print("インストール: pip install requests")

Python バージョン: 3.13.9 (tags/v3.13.9:8183fa5, Oct 14 2025, 14:09:13) [MSC v.1944 64 bit (AMD64)]
✅ openai バージョン: 1.107.2
✅ Pillow (PIL) インストール済み
✅ requests インストール済み


In [ ]:
# .env.dev を強制パースして os.environ に直接設定する
from dotenv import dotenv_values
from pathlib import Path
import os

# ノートブックから見たリポジトリルートへの相対パス (demos/openai から ../../ でルートへ)
repo_root = Path(__file__).parent.parent.parent if '__file__' in globals() else Path.cwd().parent.parent
env_path = repo_root / '.env.dev'

if env_path.exists():
    parsed = dotenv_values(env_path)
    for k, v in parsed.items():
        if v:
            os.environ[k] = v
    print(f'環境変数を読み込みました: {env_path}')
    print(f'{len(parsed)} 個のキーを設定しました。')
else:
    print(f'警告: {env_path} が見つかりません。')

# 設定状況確認
print('\n設定状況:')
print('  AZURE_OPENAI_ENDPOINT:', 'SET' if os.getenv('AZURE_OPENAI_ENDPOINT') else 'NOT SET')
print('  AZURE_OPENAI_API_KEY:', 'SET' if os.getenv('AZURE_OPENAI_API_KEY') else 'NOT SET')
print('  DALLE3_DEPLOYMENT_NAME:', os.getenv('DALLE3_DEPLOYMENT_NAME', 'NOT SET'))
print('  GPT_IMAGE_DEPLOYMENT_NAME:', os.getenv('GPT_IMAGE_DEPLOYMENT_NAME', 'NOT SET'))

環境変数を読み込みました: c:\Users\toohta\Gitrepos\AzureAIServicesDemo\.env.dev
11 個のキーを設定しました。

設定状況:
  AZURE_OPENAI_ENDPOINT: SET
  AZURE_OPENAI_API_KEY: SET
  DALLE3_DEPLOYMENT_NAME: NOT SET
  GPT_IMAGE_DEPLOYMENT_NAME: NOT SET


In [ ]:
# Azure OpenAI クライアントの初期化
from openai import AzureOpenAI
import os

endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
api_key = os.getenv("AZURE_OPENAI_API_KEY")
api_version = os.getenv("AZURE_OPENAI_API_VERSION", "2024-02-01")
# デフォルト利用モデル（優先は高品質 DALL-E 3 → 軽量 GPT-image）
dalle3_deployment = os.getenv("DALLE3_DEPLOYMENT_NAME", "dall-e-3")
gpt_image_deployment = os.getenv("GPT_IMAGE_DEPLOYMENT_NAME", "gpt-image-1")

try:
    client = AzureOpenAI(
        api_key=api_key,
        api_version=api_version,
        azure_endpoint=endpoint
    )
    print("✅ Azure OpenAI クライアント初期化完了")
    print(f"Endpoint: {endpoint}")
    print(f"API Version: {api_version}")
    print(f"DALL-E 3 Deployment: {dalle3_deployment}")
    print(f"GPT Image Deployment: {gpt_image_deployment}")
except Exception as e:
    print(f"❌ 初期化失敗: {e}")

## 2. 基本的な画像生成

**概要**: 汎用関数 `generate_image` と補助表示関数 `display_image_from_url` を定義します。DALL-E 3/2 の分岐を内部で吸収し、プロンプト → API 呼び出し → URL 取得 → 表示までの最小ループを確立します。最初のデモで生成時間と revised_prompt の違いを確認します。

In [ ]:
import requests
from PIL import Image
from io import BytesIO
from IPython.display import display
import os

# 利用中のモデル (利用者が切り替え可能)
DEFAULT_MODEL = os.getenv("DALLE3_DEPLOYMENT_NAME") or os.getenv("GPT_IMAGE_DEPLOYMENT_NAME") or "dall-e-3"

SUPPORTED_DALLE3_SIZES = {"1024x1024", "1024x1792", "1792x1024"}

def generate_image(
    prompt: str,
    model: str = None,
    size: str = "1024x1024",
    quality: str = "standard",
    style: str = "vivid"
) -> dict:
    """
    画像生成関数: DALL-E 3 / GPT-image を統合的に扱う

    Args:
        prompt: 生成したい内容を記述したテキスト
        model: 指定しない場合は DALL-E 3 優先、未設定なら GPT-image
        size: DALL-E 3 用サイズ (GPT-image は一部サイズのみ対応/標準化)
        quality: DALL-E 3 のみ (standard / hd)
        style: DALL-E 3 のみ (vivid / natural)

    Returns:
        {
          "url": 画像URL,
          "revised_prompt": 可能なら改良後プロンプト (DALL-E 3)
          "original_prompt": 入力プロンプト,
          "model_used": 実際に利用されたモデル名
        }
    """
    if model is None:
        # DALL-E 3 が使えるなら優先、なければ GPT-image
        if os.getenv("DALLE3_DEPLOYMENT_NAME"):
            model = os.getenv("DALLE3_DEPLOYMENT_NAME")
        else:
            model = os.getenv("GPT_IMAGE_DEPLOYMENT_NAME", DEFAULT_MODEL)

    is_dalle3 = "dall-e-3" in model.lower()

    try:
        if is_dalle3:
            if size not in SUPPORTED_DALLE3_SIZES:
                print(f"⚠️ サイズ {size} は DALL-E 3 非対応。1024x1024 にフォールバックします。")
                size = "1024x1024"
            response = client.images.generate(
                model=model,
                prompt=prompt,
                size=size,
                quality=quality,
                style=style,
                n=1
            )
        else:
            # GPT-image シリーズ: quality/style なし。サイズは現在 1024x1024 優先
            if size != "1024x1024":
                print(f"⚠️ GPT-image モデルはサイズ {size} 非推奨。1024x1024 にフォールバックします。")
                size = "1024x1024"
            response = client.images.generate(
                model=model,
                prompt=prompt,
                size=size,
                n=1
            )

        image_url = response.data[0].url
        revised_prompt = getattr(response.data[0], 'revised_prompt', None) if is_dalle3 else None
        return {
            "url": image_url,
            "revised_prompt": revised_prompt,
            "original_prompt": prompt,
            "model_used": model,
        }
    except Exception as e:
        print(f"エラーが発生しました: {e}")
        return None


def display_image_from_url(url: str, save_path: str = None):
    """画像URLからダウンロードして表示/保存"""
    try:
        resp = requests.get(url)
        image = Image.open(BytesIO(resp.content))
        if save_path:
            image.save(save_path)
            print(f"✅ 保存: {save_path}")
        display(image)
        return image
    except Exception as e:
        print(f"画像取得失敗: {e}")
        return None

In [ ]:
# 基本的な画像生成のデモ
prompt = "富士山と桜が美しい日本の春の風景、油絵風"

print(f"プロンプト: {prompt}\n")
print("画像を生成中...（30秒〜1分ほどかかる場合があります）\n")

result = generate_image(prompt)

if result:
    print(f"✅ 画像が生成されました！\n")
    
    if result["revised_prompt"]:
        print(f"📝 修正されたプロンプト（DALL-E 3）:")
        print(f"   {result['revised_prompt']}\n")
    
    print(f"🖼️  生成された画像:\n")
    display_image_from_url(result["url"])

## 3. さまざまなスタイルの実験

**概要**: DALL-E 3 の `style` パラメータ (`vivid` / `natural`) の違いを同一プロンプトで比較し、色彩飽和・質感・コントラストの差分を視覚的に理解します。ブランドトーン合わせやアート/写真風使い分けの指針を得ます。

In [ ]:
# スタイルバリエーションのテスト
base_prompt = "未来都市の夜景"

styles = [
    ("vivid", "鮮やかで劇的な表現"),
    ("natural", "自然でリアルな表現")
]

print(f"ベースプロンプト: {base_prompt}\n")
print("2つの異なるスタイルで画像を生成します...\n")

for style, description in styles:
    print(f"{'='*60}")
    print(f"スタイル: {style} ({description})")
    print(f"{'='*60}\n")
    
    result = generate_image(base_prompt, style=style)
    
    if result:
        if result["revised_prompt"]:
            print(f"修正されたプロンプト: {result['revised_prompt'][:100]}...\n")
        display_image_from_url(result["url"])
        print("\n")

## 4. 画質とサイズの比較

**概要**: 品質パラメータ `quality=standard/hd` の解像感・細部再現性差を確認しコスト最適化判断材料を得ます。さらにサイズパラメータ (正方形/縦長/横長) により用途別アスペクト比選択の効果を確認します。SNS投稿・ヘッダー・縦広告などのレイアウト適合を検証します。

In [ ]:
# 画質比較（DALL-E 3 のみ）
prompt = "宇宙ステーションから見た地球、フォトリアリスティック"

qualities = [
    ("standard", "標準品質（高速・低コスト）"),
    ("hd", "HD品質（高品質・高コスト）")
]

print(f"プロンプト: {prompt}\n")
print("異なる品質設定で画像を生成します...\n")

for quality, description in qualities:
    print(f"{'='*60}")
    print(f"品質: {quality} ({description})")
    print(f"{'='*60}\n")
    
    result = generate_image(prompt, quality=quality)
    
    if result:
        display_image_from_url(result["url"])
        print("\n")

In [ ]:
# サイズバリエーション（DALL-E 3）
prompt = "抽象的なアートワーク、カラフルな幾何学模様"

sizes = [
    ("1024x1024", "正方形"),
    ("1024x1792", "縦長（ポートレート）"),
    ("1792x1024", "横長（ランドスケープ）")
]

print(f"プロンプト: {prompt}\n")
print("異なるサイズで画像を生成します...\n")

for size, description in sizes:
    print(f"{'='*60}")
    print(f"サイズ: {size} ({description})")
    print(f"{'='*60}\n")
    
    result = generate_image(prompt, size=size)
    
    if result:
        display_image_from_url(result["url"])
        print("\n")

## 5. プロンプトエンジニアリングのベストプラクティス

**概要**: 効果的なプロンプト要素（詳細な構成要素・スタイル・光源・感情・構図）を例示し、曖昧・冗長・矛盾・不適切コンテンツ回避の観点を整理します。DALL-E 3 の自動改善を前提にしつつ、意図再現性を高める最小十分プロンプト作成ノウハウを獲得します。

In [ ]:
# 効果的なプロンプトの例
prompts = [
    {
        "name": "詳細な説明",
        "prompt": "小さな村の中央にある古い石造りの噴水。周囲には色とりどりの花が咲き、背景には緑豊かな丘陵地帯が広がる。温かい夕日の光が全体を照らしている。水彩画風。",
        "tips": "具体的な要素、色、光、スタイルを指定"
    },
    {
        "name": "アートスタイル指定",
        "prompt": "サイバーパンクな東京の街並み、ネオンサイン、雨に濡れた路面。ブレードランナー風のシネマティックな雰囲気。",
        "tips": "既知のスタイルや映画を参照"
    },
    {
        "name": "感情と雰囲気",
        "prompt": "静かで穏やかな森の中の小道。朝霧が立ち込め、木漏れ日が差し込む。平和で瞑想的な雰囲気。",
        "tips": "感情や雰囲気のキーワードを含める"
    }
]

for example in prompts:
    print(f"\n{'='*60}")
    print(f"例: {example['name']}")
    print(f"{'='*60}")
    print(f"\nプロンプト: {example['prompt']}")
    print(f"\n💡 ヒント: {example['tips']}\n")
    
    result = generate_image(example['prompt'])
    
    if result:
        if result["revised_prompt"]:
            print(f"\n修正されたプロンプト:\n{result['revised_prompt']}\n")
        display_image_from_url(result["url"])
        print("\n")

## 6. 画像の保存とバッチ処理

**概要**: 単発生成を運用へ組み込むため、生成画像+メタデータ(プロンプト/revised_prompt)をファイルとして保存し再現性を確保します。バッチ処理で複数プロンプトを連続投入し、命名規則とロギングを整える基盤を示します。

In [ ]:
from datetime import datetime
import os

# 画像保存用のディレクトリを作成
output_dir = Path("generated_images")
output_dir.mkdir(exist_ok=True)

def generate_and_save_image(prompt: str, filename: str = None, **kwargs) -> str:
    """
    画像を生成して保存する
    
    Args:
        prompt: 画像生成プロンプト
        filename: 保存ファイル名（指定しない場合は自動生成）
        **kwargs: generate_image に渡す追加パラメータ
        
    Returns:
        保存したファイルのパス
    """
    result = generate_image(prompt, **kwargs)
    
    if not result:
        return None
    
    # ファイル名の生成
    if filename is None:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"dalle_{timestamp}.png"
    
    save_path = output_dir / filename
    
    # 画像をダウンロードして保存
    response = requests.get(result["url"])
    image = Image.open(BytesIO(response.content))
    image.save(save_path)
    
    print(f"✅ 画像を保存しました: {save_path}")
    
    # メタデータを保存
    metadata_path = save_path.with_suffix('.txt')
    with open(metadata_path, 'w', encoding='utf-8') as f:
        f.write(f"Original Prompt:\n{result['original_prompt']}\n\n")
        if result['revised_prompt']:
            f.write(f"Revised Prompt:\n{result['revised_prompt']}\n")
    
    print(f"✅ メタデータを保存しました: {metadata_path}")
    
    display(image)
    return str(save_path)

# デモ: 画像を生成して保存
prompt = "魔法使いの書斎、古い本が積み重なり、魔法の光が漂う。ファンタジーアート風。"
print(f"プロンプト: {prompt}\n")

saved_path = generate_and_save_image(prompt, filename="wizard_library.png")
print(f"\n保存先: {saved_path}")

In [ ]:
# バッチ生成のデモ
batch_prompts = [
    "春の桜が満開の京都の寺院",
    "夏のビーチでサーフィンをする人々",
    "秋の紅葉に包まれた山道",
    "冬の雪景色の中の温泉旅館"
]

print("バッチ処理で複数の画像を生成します...\n")

for i, prompt in enumerate(batch_prompts, 1):
    print(f"\n{'='*60}")
    print(f"画像 {i}/{len(batch_prompts)}")
    print(f"{'='*60}")
    print(f"プロンプト: {prompt}\n")
    
    filename = f"season_{i:02d}.png"
    saved_path = generate_and_save_image(prompt, filename=filename)
    
    if saved_path:
        print(f"✅ 完了: {filename}\n")
    else:
        print(f"❌ エラー: {filename}\n")

print(f"\n✅ バッチ処理が完了しました。{output_dir} フォルダを確認してください。")

## 7. プロンプトのヒントとベストプラクティス

**概要**: 具体化・スタイル明示・光/色/構図/感情の付与に関する詳細チェックリストを列挙し、失敗例（曖昧/矛盾/冗長）の回避策を体系化します。コスト最適化の観点 (standard→hd 段階的検証) も合わせて整理しチーム共有可能なガイドライン化を行います。

### 効果的なプロンプトの書き方

#### ✅ 推奨事項

1. **具体的に記述する**
   - ❌ "美しい風景"
   - ✅ "夕暮れ時の富士山、湖に映る逆さ富士、桜の木が前景に"

2. **スタイルを明示する**
   - "油絵風"、"水彩画風"、"フォトリアリスティック"、"アニメ風"
   - "ピクサー風"、"ゴッホ風"、"シネマティック"

3. **光と色を指定する**
   - "温かい夕日の光"、"柔らかい朝の光"、"ネオンの輝き"
   - "パステルカラー"、"ビビッドな色彩"、"モノクロ"

4. **構図を考慮する**
   - "クローズアップ"、"鳥瞰図"、"低い角度から"
   - "中央配置"、"黄金比に従って"

5. **雰囲気や感情を加える**
   - "平和な"、"ドラマチックな"、"神秘的な"、"活気のある"

#### ❌ 避けるべきこと

1. **曖昧すぎる表現**
   - "良い感じの画像" → 具体性がない

2. **矛盾する指示**
   - "リアルでファンタジーな" → スタイルが矛盾

3. **過度に長いプロンプト**
   - 重要な要素に絞る（DALL-E は自動で補完・調整する）

4. **不適切なコンテンツ**
   - 暴力的、性的、差別的な内容は禁止
   - 著名人の名前の使用は制限される

### DALL-E 3 の特徴

- **自動プロンプト改善**: 入力プロンプトを自動的に詳細化・改善
- **高品質**: DALL-E 2 より詳細で正確な画像生成
- **テキスト描画**: 画像内のテキストの精度が向上
- **安全性**: より厳格なコンテンツフィルタリング

### コスト最適化のヒント

1. **standard 品質から始める**（必要な場合のみ hd を使用）
2. **適切なサイズを選択**（用途に応じて）
3. **プロンプトを事前にテスト**（小さなサイズで試す）
4. **生成回数を計画的に**（バッチ処理で効率化）

## 8. 実践的な使用例: マーケティング素材の生成

**概要**: 用途別 (広告 / ブログヘッダー / SNS 正方形) にアスペクト比と品質を最適化し、再利用可能なシナリオテンプレートを提示します。実運用で頻出するパラメータ組み合わせの標準化と命名規則の例を示します。

In [ ]:
# マーケティング素材生成の例
marketing_scenarios = [
    {
        "name": "製品広告",
        "prompt": "最新のスマートフォンを持つ若者、都会的な背景、現代的でスタイリッシュ、商品写真風、高品質",
        "size": "1024x1792"  # 縦長（SNS投稿用）
    },
    {
        "name": "ブログヘッダー",
        "prompt": "テクノロジーとイノベーションを表現する抽象的なデジタルアート、青とオレンジのグラデーション、未来的",
        "size": "1792x1024"  # 横長（ヘッダー用）
    },
    {
        "name": "SNS投稿",
        "prompt": "コーヒーカップとノートパソコンが置かれた洗練されたデスク、朝の光が差し込む、ライフスタイル写真風",
        "size": "1024x1024"  # 正方形（Instagram用）
    }
]

print("マーケティング素材を生成します...\n")

for scenario in marketing_scenarios:
    print(f"\n{'='*60}")
    print(f"用途: {scenario['name']}")
    print(f"{'='*60}")
    print(f"プロンプト: {scenario['prompt']}")
    print(f"サイズ: {scenario['size']}\n")
    
    filename = f"marketing_{scenario['name'].replace(' ', '_').lower()}.png"
    saved_path = generate_and_save_image(
        scenario['prompt'],
        filename=filename,
        size=scenario['size'],
        quality="hd"  # マーケティング素材なので高品質
    )
    
    if saved_path:
        print(f"\n✅ {scenario['name']} を保存しました\n")

## まとめ

このノートブックでは Azure OpenAI の **DALL-E 3** と **GPT-image-1** を用いた画像生成を実装しました。

### 実装した機能
1. ✅ 統合画像生成関数 (モデル自動切替)
2. ✅ スタイル / 品質 / サイズ比較 (DALL-E 3 特有)
3. ✅ プロンプトエンジニアリング例
4. ✅ 画像保存 + メタデータ (再現性)
5. ✅ バッチ処理 (季節別・用途別)
6. ✅ マーケティング素材テンプレート

### モデル比較 (要約)
| 特徴 | DALL-E 3 | GPT-image-1 |
|------|----------|-------------|
| 品質 | 高精細 | 高速・自然 |
| プロンプト改善 | あり (revised_prompt) | なし (入力忠実) |
| スタイル指定 | vivid / natural | なし |
| 品質指定 | standard / hd | なし |
| サイズ選択 | 3種 (正方/縦/横) | 主に 1024x1024 |
| コスト | 高 | 低～中 |
| 推奨用途 | 高付加価値クリエイティブ | 大量生成 / プロトタイプ |

### ベストプラクティス (再掲)
- 重要要素を箇条書きで整理 (被写体・背景・スタイル・光源・雰囲気)
- PoC / 大量生成は GPT-image → 品質検証後 DALL-E 3 に昇格
- メタデータ保存で品質差異分析 (revised vs original)
- コスト最適化: 最初は standard と単一サイズ、確定後 hd / サイズ展開
- 不適切コンテンツとブランドガイドライン遵守

### 次のステップ
- Image Edit / Variation API 追加
- 生成画像の自動品質評価 (モデルベースフィルタリング)
- Azure Storage / CDN 配信最適化
- プロンプトテンプレートのバージョン管理と A/B テスト

> 運用では使用モデル・品質・サイズ・CTR 等をログ化し、データドリブンに最適化サイクルを回してください。